In [14]:
import pandas as pd
import numpy as np

df = pd.read_csv('nba_subset_raw.csv')

In [15]:
df.head()

,Salary,Age,Player_Efficiency_Rating,Blocks,Turnovers_Pct,Steals,Assists_Pct,Games_Started,Games_Played,Total_Rebounds,Field_Goals,Defensive_Box_Plus_Minus,Defensive_Rebound_Pct,Usage_Pct,Free-Throws
0,1312611,24,8.4,13,16.4,1,3.8,0,22,36,17,1.9,20.9,17.6,9
1,2116955,32,9.5,9,17.2,25,20.7,0,65,69,121,-2.6,6.3,19.2,32
2,5504420,21,14.4,40,8.5,64,10.5,72,80,405,393,-0.4,14.1,20.1,156
3,365289,22,-2.2,0,0.0,0,22.6,0,5,3,0,-2.1,19.5,12.9,1
4,2022240,25,14.4,7,11.4,8,8.0,0,18,33,23,0.7,21.0,23.1,14


Create 8 classes of salary and bin them

In [68]:
df['cut'] = pd.qcut(df['Salary'], 2)
df['sal_bin'] = pd.qcut(df['Salary'], 2, labels=['1','2']).astype('int32')

In [72]:
df.head()

,Salary,Age,Player_Efficiency_Rating,Blocks,Turnovers_Pct,Steals,Assists_Pct,Games_Started,Games_Played,Total_Rebounds,Field_Goals,Defensive_Box_Plus_Minus,Defensive_Rebound_Pct,Usage_Pct,Free-Throws,cut,sal_bin
0,1312611,24,8.4,13,16.4,1,3.8,0,22,36,17,1.9,20.9,17.6,9,"(17223.999, 3343830.0]",1
1,2116955,32,9.5,9,17.2,25,20.7,0,65,69,121,-2.6,6.3,19.2,32,"(17223.999, 3343830.0]",1
2,5504420,21,14.4,40,8.5,64,10.5,72,80,405,393,-0.4,14.1,20.1,156,"(3343830.0, 34682550.0]",2
3,365289,22,-2.2,0,0.0,0,22.6,0,5,3,0,-2.1,19.5,12.9,1,"(17223.999, 3343830.0]",1
4,2022240,25,14.4,7,11.4,8,8.0,0,18,33,23,0.7,21.0,23.1,14,"(17223.999, 3343830.0]",1


Random forest Regression (without binning salary)

In [42]:
from sklearn.model_selection import train_test_split
other = ['Salary', 'cut', 'sal_bin']
target_column = ['Salary'] 
predictors = list(set(list(df.columns))-set(other))

X = df[predictors]
X.head()
X = np.array(df[predictors])
y = np.array(df[target_column])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)

In [43]:
df[predictors].head()

,Player_Efficiency_Rating,Turnovers_Pct,Age,Free-Throws,Games_Started,Assists_Pct,Blocks,Total_Rebounds,Usage_Pct,Steals,Field_Goals,Defensive_Box_Plus_Minus,Defensive_Rebound_Pct,Games_Played
0,8.4,16.4,24,9,0,3.8,13,36,17.6,1,17,1.9,20.9,22
1,9.5,17.2,32,32,0,20.7,9,69,19.2,25,121,-2.6,6.3,65
2,14.4,8.5,21,156,72,10.5,40,405,20.1,64,393,-0.4,14.1,80
3,-2.2,0.0,22,1,0,22.6,0,3,12.9,0,0,-2.1,19.5,5
4,14.4,11.4,25,14,0,8.0,7,33,23.1,8,23,0.7,21.0,18


In [50]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

rf.fit(X_train, y_train.ravel())

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=1000,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [63]:
pred = rf.predict(X_test)
errors = abs(pred - y_test)

print('Mean Absolute Error:', round(np.mean(errors), 2), 'dollars')

Mean Absolute Error: 7104694.28 dollars


Random Forest Classifer (with bins)

In [69]:
from sklearn.model_selection import train_test_split
other = ['Salary', 'cut', 'sal_bin']
target_column = ['sal_bin'] 
predictors = list(set(list(df.columns))-set(other))

X = df[predictors]
X.head()
X = np.array(df[predictors])
y = np.array(df[target_column])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)

In [70]:
from sklearn.ensemble import RandomForestClassifier

clf=RandomForestClassifier(n_estimators=1000)

clf.fit(X_train, y_train.ravel())

y_pred=clf.predict(X_test)

In [71]:
from sklearn import metrics

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7808219178082192
